In [1]:
import neo4j
import osmnx as ox
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_osm import poidf2rows
from utils_neo4j import init, poi_insert_query

In [41]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [2]:
# center point hamburg
center_point = (53.5511, 9.9937)  # Latitude and Longitude of Hamburg

# Define the tags for different POIs you're interested in
tags = {
    'amenity': ['restaurant', 'cafe', 'fast_food', 'food_court', 'pub', 'bar'],
}


In [5]:
# Fetch the POIs
pois = ox.features_from_point(center_point, tags=tags, dist=15000)
pois = pois.reset_index()

In [11]:
pd.set_option('display.max_columns', None)
pois.head(3)


,element,id,geometry,addr:city,addr:housenumber,addr:postcode,addr:street,amenity,bar,branch,brand,brand:wikidata,cuisine,name,opening_hours,operator,outdoor_seating,phone,reservation,start_date,toilets:wheelchair,website,website:menu,wheelchair,check_date,check_date:opening_hours,opening_hours:signed,smoking,internet_access,takeaway,addr:country,contact:phone,delivery,diet:vegan,diet:vegetarian,fax,microbrewery,payment:american_express,payment:cash,payment:girocard,payment:mastercard,payment:visa,capacity,email,brand:wikipedia,drive_through,indoor_seating,internet_access:fee,payment:coins,payment:contactless,payment:credit_cards,payment:debit_cards,payment:visa_debit,level,contact:email,contact:facebook,contact:instagram,indoor,ref,contact:website,addr:housename,alt_name,contact:twitter,description,diet:halal,diet:meat,brewery,source,source_date,toilets,name:de,stars,shop,check_date:diet:vegetarian,name:zh,note,self_service,opening_hours:covid19,min_age,real_fire,addr:suburb,bettundbike,organic,tourism,air_conditioning,diet:kosher,ref:vatin,changing_table,payment:cryptocurrencies,payment:electronic_purses,toilets:access,lunch,lunch:menu,lunch:menu:choices,lunch:menu:url,lunch:saladbar,source:name,payment:maestro,contact:fax,payment:notes,wheelchair:description,entrance,official_name,old_name,diet:non-vegetarian,payment:v_pay,contact:tripadvisor,wikidata,check_date:smoking,service:electricity,mobile,food,opening_hours:kitchen,diet:gluten_free,diet:lactose_free,payment:diners_club,payment:discover_card,created_by,charge,fee,breakfast,kids_area,kids_area:outdoor,toilets:menstrual_products,payment:jcb,payment:visa_electron,payment:edenred_card,payment:sodexo,short_name,surveillance,check_date:diet:vegan,description:en,brewery:Pils,wheelchair:description:de,cocktails,diet:pescetarian,payment:giropay,theme,opening_hours:url,drive_in,sport,name:en,pay_tv,diet:mediterranean,diet:organic,fair_trade,internet_access:ssid,payment:cards,highchair,darts,not:brand:wikidata,access:covid19,drive_through:covid19,takeaway:covid19,url,addr:city:fa,name:signed,drink:club-mate,internet_access:operator,facebook,lgbtq,outdoor_seating:capacity,wifi:limit,service_times,dog,access,diet:fish,internet_access:password,payment:ep_geldkarte,fixme,payment:alipay,payment:unionpay,payment:amex,payment:apple_pay,payment:ec,payment:google_pay,disused:name,fixme:toilets:wheelchair,fixme:wheelchair,long_name,lastcheck,payment:description,leisure,drink:coffee,female:signed,male:signed,man_made,delivery:covid19,kicker,layer,full_name,toilets:unisex,addr:floor,wheelchair:description:en,outdoor,contact:youtube,wikipedia,speciality,contact:mobile,safety:mask:covid19,description:de,old_website,owner,atm,payment:unionpay_debit,name:ru,pets,image,contact:pinterest,cafe,wifi,name:old,note:vacant,stroller,disused:shop,name:vi,contact:website:menu,frontcooking,check_date:toilets:wheelchair,payment:edenred_voucher,post_office,post_office:brand,post_office:brand:wikidata,post_office:service_provider,note:opening_hours,check_date:diet:halal,check_date:diet:kosher,smoking:cannabis,happycow:id,survey:date,note:de,operator:type,name:it,was:website,smoking:outside,noname,diet:lacto_vegetarian,description:payment,music,loc_name,real_ale,note:cuisine,description:toilets,check_date:currency:XBT,currency:XBT,payment:lightning,payment:lightning_contactless,payment:onchain,origin,live_music,kids,name:es,door,ownership,contact:mail,node,payment:visa_contactless,smoker,darts:type,diet:healthy,check_date:website,disused:website,note:website,toilets:disposal,toilets:fee,toilets:position,payment:app,opening_date,shisha,lgbtq:trans,restaurant,check_date:internet_access,drink:oat_milk,drink:soy_milk,strapline,name:fa,description:covid19,drink:beer,drink:lemonade,drink:tea,drink:water,website:map,addr:place,changing_table:count,changing_table:location,drink:wine,contact:yelp,surveillance:type,payment:cheque,source:opening_hours,fast_food,seats,name:uk,craft,payment:mastercard_contactless,kids_ar

In [20]:
import numpy as np
(~pois['description:de'].isna()).sum()

np.int64(3)

In [26]:
# description based on: 'description:de', 'description' 
# tags based on: 'amenity', 'bar', 'branch', 'brand', 'cuisine'
tagcols = ['amenity', 'bar', 'branch', 'cuisine']
impcols = ['id', 'geometry', 'name', 'addr:street', 'addr:housenumber', 'addr:postcode', 'description:de', 'description', *tagcols]

In [27]:
pois_impcols = pois[impcols]

In [28]:
mask = pois_impcols.name.isna()
pois_impcols = pois_impcols[~mask]

In [29]:
pois_impcols['geometry'] = pois_impcols['geometry'].apply(simplify_to_centroid_if_small)

In [30]:
# step 1
# combine col description:de and description, use description:de 
# if available else description, if both are not available use empty string
pois_impcols['description'] = pois_impcols['description:de'].combine_first(pois_impcols['description'])
pois_impcols.drop(columns=['description:de'], inplace=True)

In [31]:
# step 2
# 'amenity', 'bar', 'branch', 'cuisine'
# put values in a col called tags which is list of strings but only if the value is not null or empty string
# remove the columns from pois_impcols
# Modified code to add column name prefixes to tag values
pois_impcols['tags'] = pois_impcols[tagcols].apply(
    lambda row: [f"{col}:{val}" for col, val in zip(row.index, row) 
                if pd.notna(val) and val != ''], 
    axis=1
)
pois_impcols.drop(columns=tagcols, inplace=True)

In [32]:
# drop columns where geometry is empty 
pois_impcols = pois_impcols[pois_impcols['geometry'].notna()]
# drop columns where name and description and tags is empty
mask = pois_impcols.tags.apply(lambda x: len(x) > 0) | pois_impcols.name.notna() | pois_impcols.description.notna()
pois_impcols = pois_impcols[mask]

In [33]:
# rename col id to osmid
pois_impcols.rename(columns={'id': 'osmid'}, inplace=True)

In [34]:
pois_impcols.head()

,osmid,geometry,name,addr:street,addr:housenumber,addr:postcode,description,tags
0,25745452,POINT (10.11644 53.60807),Schweinske,August-Krogmann-Straße,25,22159,NaN,"[amenity:restaurant, bar:yes, branch:Farmsen, ..."
1,25776282,POINT (10.04363 53.57887),Most wanted Burger,NaN,NaN,NaN,NaN,"[amenity:fast_food, cuisine:burger]"
2,25903669,POINT (9.86603 53.53385),Bar 380,Finkenwerder Norderdeich,18a,21129,NaN,[amenity:pub]
3,55104007,POINT (9.90304 53.60673),Block House,NaN,NaN,NaN,NaN,"[amenity:restaurant, cuisine:steak_house]"
4,68424433,POINT (10.16384 53.64927),deFries express,Claus-Ferck-Straße,6,NaN,NaN,[amenity:fast_food]


In [35]:
# filter rows where geometry dont is point
notpois_impcols = pois_impcols[pois_impcols['geometry'].apply(lambda x: x.geom_type != 'Point')]
pois_impcols = pois_impcols[pois_impcols['geometry'].apply(lambda x: x.geom_type == 'Point')]

In [36]:
pois_impcols.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 5170 entries, 0 to 5220
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   osmid             5170 non-null   int64   
 1   geometry          5170 non-null   geometry
 2   name              5170 non-null   object  
 3   addr:street       3077 non-null   object  
 4   addr:housenumber  3040 non-null   object  
 5   addr:postcode     2771 non-null   object  
 6   description       138 non-null    object  
 7   tags              5170 non-null   object  
dtypes: geometry(1), int64(1), object(6)
memory usage: 363.5+ KB


In [37]:
len(notpois_impcols)

0

In [38]:
rows = poidf2rows(pois_impcols)

In [39]:
rows

[{'osmid': 25745452,
  'name': 'Schweinske',
  'addr_street': 'August-Krogmann-Straße',
  'addr_housenumber': '25',
  'addr_postcode': '22159',
  'description': None,
  'tags': ['amenity:restaurant',
   'bar:yes',
   'branch:Farmsen',
   'cuisine:burger;escalope;german'],
  'longitude': 10.1164404,
  'latitude': 53.6080733},
 {'osmid': 25776282,
  'name': 'Most wanted Burger',
  'addr_street': None,
  'addr_housenumber': None,
  'addr_postcode': None,
  'description': None,
  'tags': ['amenity:fast_food', 'cuisine:burger'],
  'longitude': 10.0436269,
  'latitude': 53.5788725},
 {'osmid': 25903669,
  'name': 'Bar 380',
  'addr_street': 'Finkenwerder Norderdeich',
  'addr_housenumber': '18a',
  'addr_postcode': '21129',
  'description': None,
  'tags': ['amenity:pub'],
  'longitude': 9.8660342,
  'latitude': 53.5338482},
 {'osmid': 55104007,
  'name': 'Block House',
  'addr_street': None,
  'addr_housenumber': None,
  'addr_postcode': None,
  'description': None,
  'tags': ['amenity:rest

In [42]:
# Execute the query
with driver.session() as session:
    result = session.run(poi_insert_query, rows=rows)
    print(f"Imported {result.single()['total']} POIs")

driver.close()

Imported 5170 POIs
